# Init

In [1]:
import sys
import os
import json
import time
import re

from tqdm import tqdm

cwd = os.getcwd()
os.chdir(cwd)
sys.path.append('tools')

import chat
import parse_data
import sql

In [2]:
data_dir = os.path.join(cwd, 'exp_comparison')

In [3]:
def extract_questions(fpath: str):
    """
    Extracts Question ID and raw_question from the input file and returns 
    a dictionary where keys are the integer forms of multi-digit Question IDs and values are the raw questions.
    """

    with open(fpath, 'r') as f:
        text = ''.join(f.readlines())

    reg_pattern = re.compile('v\d\.\d.\d')
    version = re.search(reg_pattern, fpath)[0]

    question_dict = {}  # Initialize an empty dictionary to store the results
    
    # Regular expression to match the Question ID and capture its integer part (single or multi-digit)
    question_id_pattern = re.compile(r"Question ID: .*?(\d+)")
    
    # Regular expression to match the raw_question value
    raw_question_pattern = re.compile(r'"raw_question":\s*"(.*?)"')
    
    # Find all Question IDs in the text
    question_ids = question_id_pattern.findall(text)
    
    # Find all raw_question values in the text
    raw_questions = raw_question_pattern.findall(text)
    
    # Iterate over both lists and populate the dictionary
    for question_id, raw_question in zip(question_ids, raw_questions):
        question_dict[int(question_id)] = raw_question  # Convert multi-digit Question ID to integer and add to dictionary
    
    return {version: question_dict}  # Return the final dictionary

def find_cases(stage = 'stage_1', agent_name = 'table_finder'):

    tag = f'{stage}-{agent_name}'
    files = [i for i in sorted(os.listdir(data_dir)) if tag in i]
    # print(os.listdir(data_dir))
    # print(tag, files)
    res = {}

    for fname in files:
        fpath = os.path.join(data_dir, fname)
        res.update(extract_questions(fpath))

    return res

# Stage-1

In [4]:
stage = 'stage_1'

## Table Finder 

In [5]:
agent_name = 'table_finder'

### Craft Prompt

In [6]:
system_prompt = ""

prompt_dir = os.path.join(cwd, 'prompt')
version = 'v2.6.2'
fname = f'table_finder-stage_1-{version}.md'
prompt_fpath = os.path.join(prompt_dir, fname)

with open(prompt_fpath, 'r') as f:
    prompt_template = ''.join(f.readlines())


def parse_database_and_table(query: str) -> dict:
    """
    Parse the given SQL query to return the database and table names in a dictionary.
    
    Args:
    - query (str): The SQL query to parse.
    
    Returns:
    - dict: A dictionary with 'database' and 'table' keys.
    """

    pattern = r'FROM\s+([a-zA-Z0-9_]+)\.([a-zA-Z0-9_]+)'
    match = re.search(pattern, query, re.IGNORECASE)
    
    if match:
        database = match.group(1)
        table = match.group(2)
        return {'database': database, 'table': table}
    
    return {}


def make_prompt(query: str, ner: dict) -> str:

    """
    ner_res: content from the stage_1
    """

    prompt = prompt_template + query

    # ner_result is None
    if not ner['result']:
        return prompt

    ner_content = {}
    ner_content.update(ner['result'][0])

    sql_res = ner['sql']

    for k, v in sql_res.items():
        for j in v:
            if not j['result']:
                continue

            sql_query = j['query']
            # add database and table
            ner_content.update(parse_database_and_table(sql_query))
            ner_content['ner_result'] = j['result']

    # add NER result

    ner_str = f"\n\n### **Name Entity Recognition Result**\n```json\n{json.dumps(ner_content, ensure_ascii=False,indent=2)}\n```"

    prompt += ner_str
    
    return prompt

In [7]:
question_path = os.path.join(cwd, 'answer_tmp' + os.sep + 'stage_1-glm_4_plus-ner-sql-HF.json')

questions = parse_data.read_json(question_path)

### Cases

In [8]:
cases = find_cases(stage = 'stage_1', agent_name = 'table_finder')

print(json.dumps(cases, ensure_ascii=False, indent=2))

{
  "v2.3.0": {
    "4": "互联网金融属于科技概念的什么分支？这个概念的英文名称是什么？",
    "5": "互联网金融属于科技概念的什么分支？这个概念的英文名称是什么？",
    "12": "化工纳入过多少个子类概念？",
    "13": "化工纳入过多少个子类概念？",
    "14": "今天是2020年6月24日，阅文集团近一个月最高价是（保留2位小数）？",
    "24": "今天是2020年6月24日，阅文集团近一个月最高价是（保留2位小数）？",
    "30": "今天是2020年10月27日，当日收盘价第3高的港股是？(以下都回答简称)",
    "34": "今天是2020年10月27日，当日收盘价第3高的港股是？(以下都回答简称)",
    "53": "2019-09-02，当日收盘价最高的港股是？(以下都回答简称)",
    "55": "2019-09-02，当日收盘价最高的港股是？(以下都回答简称)",
    "58": "山东药玻2020年发布的19年年报的大股东是谁",
    "63": "山东药玻2020年发布的19年年报的大股东是谁？",
    "64": "李一硕一共管理了多少支基金",
    "72": "李一硕一共管理了多少支基金？",
    "74": "山东国瓷功能材料股份有限公司2021年9月23日开盘价是多少？",
    "85": "山东国瓷功能材料股份有限公司2021年9月23日开盘价是多少？",
    "87": "水晶光电实施完成的员工持股计划有几个？",
    "88": "水晶光电实施完成的员工持股计划有几个？",
    "91": "截止2021-06-17上海建工的近一周成交金额（万元）是多少？",
    "96": "截止2021-06-17上海建工的近一周成交金额（万元）是多少？",
    "97": "2021年1月11日，正常交易且跳空低开的股票一共有几只？",
    "98": "2021年1月11日，正常交易且跳空低开的股票一共有几只？",
    "99": "最新更新的2019年度报告中，机构持有无限售流通A股数量合计最多的公司简称是？",
    "100": "最新更新的2019年度报告中，机构持有无限售流

In [9]:
num = 16
num -= 1

print(make_prompt(questions[num]['team'][0]['question'], questions[num]['ner']['stage_1']))

## **Database-Table Schema**

### ConstantDB
**数据库简介**：
作为常量库，包含各种基础性、标准化的参照表，如证券主表（记录股票、基金、债券等基础信息）、系统常量表（各种常量分类）、国家城市代码表、交易日表等。

**典型应用**：
- 获取证券的通用代码、简称、上市地点等；  
- 查询某一天是否为交易日，以及周、月、季、年的最后一个交易日。  
- 根据地区代码查询对应的国家或城市名称。

|库名中文|库名英文|表英文|表中文|表描述|数据范围|信息来源|
|---|---|---|---|---|---|---|
| 常量库| ConstantDB| HK_SecuMain| 港股证券主表 | 记录港股单个证券品种的简称、上市交易所等基础信息。 || 
| 常量库 | ConstantDB| US_SecuMain| 美股证券主表 | 记录美国等境外市场单个证券品种的简称、上市交易所等基础信息。|||
| 常量库 | ConstantDB| SecuMain | 证券主表 | 记录A股单个证券品种（股票、基金、债券）的代码、简称、上市交易所等基础信息。 |||
| 常量库 | ConstantDB| CT_SystemConst | 系统常量表 | 本表收录数据库中各种常量值的具体分类和常量名称描述。 |||
| 常量库 | ConstantDB| LC_AreaCode | 国家城市代码表 | 本表收录世界所有国家层面的数据信息和我国不同层级行政区域的划分信息。 |||
| 常量库 | ConstantDB | QT_TradingDayNew | 交易日表(新) | 本表收录各个市场的交易日信息，包括每个日期是否是交易日，是否周、月、季、年最后一个交易日。 |||

### AStockBasicInfoDB
**数据库简介**：
主要收录 A 股上市公司的基础信息，如公司概况、名称变更、经营范围与行业变更等，帮助用户快速了解公司基本背景。

**典型应用**：
- 查询公司成立背景、联系方式、注册资本等基础资料。  
- 追踪公司历次名称变更及其原因。  
- 获取公司经营范围的历史变化，用于分析企业业务重心。

|库名中文|库名英文|表英文|表中文|表描述|数据范围|信息来源|
|---|---|---|

## SQL Generator

In [10]:
agent_name = 'sql_generator'

### Craft Prompt

In [11]:
system_prompt = ""

prompt_dir = os.path.join(cwd, 'prompt')
version = 'v1.1.0'
fname = f'sql_generator-stage_1-{version}.md'
prompt_fpath = os.path.join(prompt_dir, fname)

with open(prompt_fpath, 'r') as f:
    prompt_template = ''.join(f.readlines())

def make_prompt(data: dict) -> str:

    prompt = prompt_template

    # 
    table_finder_res = data['table_finder']['stage_1'][0]['data_source'][0]
    # del table_finder_res['question']
    table = table_finder_res['table']
    table_finder_res = json.dumps(table_finder_res, ensure_ascii=False, indent=2)
    reg_p = re.compile('<Database and Table>')
    prompt = re.sub(reg_p, table_finder_res, prompt)

    # 
    table_fname = f'{table}-with_table_name.md'
    table_dir = os.path.join(cwd, 'data' + os.sep + 'table-column')
    table_fpath = os.path.join(table_dir, table_fname)
    with open(table_fpath,'r') as f:
        table_schema = ''.join(f.readlines())
    reg_p = re.compile('<Table-Column Schema>')
    prompt = re.sub(reg_p, table_schema, prompt)

    # 
    if data['ner']['stage_1']['result']:
        ner_res = [i for i in data['ner']['stage_1']['sql'].values() if i][0]
        ner_res = [i['result'] for i in ner_res if i['result']][0][0]
        ner_res = json.dumps(ner_res, ensure_ascii=False, indent=2)
        reg_p = re.compile('<Background Knowledge>')
        prompt = re.sub(reg_p, ner_res, prompt)
    else:
        reg_p = re.compile('<Background Knowledge>')
        prompt = re.sub(reg_p, '', prompt)
        reg_p = re.compile('## Background Knowledge')
        prompt = re.sub(reg_p, '', prompt)

    # replace query
    query = data['team'][0]['question']
    reg_p = re.compile('<Current Query>')
    prompt = re.sub(reg_p, query, prompt)

    return prompt

In [12]:
fname = 'stage_1-glm_4_plus-table_finder-v2.6.2.json'

question_path = os.path.join(cwd, 'answer_tmp' + os.sep + fname)

questions = parse_data.read_json(question_path)

### Cases

In [13]:
failed = {
    "v1.0.0":
    {
        2: "今天是2021年12月24日，创近半年新高的股票有几只？",
        13: "今天是2020年10月27日，当日收盘价第3高的港股是？(以下都回答简称)",
        63: "最新更新的2019年度报告中，机构持有无限售流通A股数量合计最多的公司简称是？",
        64: "最新更新的2021年度报告中，机构持有无限售流通A股数量合计最多的公司简称是？",
        74: "天弘增利短债C的基金管理人是谁？",
        84: "永泰能源在2020年发生了几次业务范围变更？请列出每次变更的具体日期（xxxx-xx-xx的格式）",
        97: "深科技在2021年12月24日的交易数据如何?具体包括收盘价、成交量、换手率，保留2位小数。"
    }
}

cases = find_cases(stage='stage_1', agent_name='sql_generator')

print(json.dumps(cases, ensure_ascii=False, indent=2))

{}


In [14]:
num = 97
num -= 1

print(make_prompt(questions[num]))

## Task Description

You are an SQL expert tasked with generating MySQL queries based on the provided Database-Table Information, Table-Column Schema, and Background Knowledge. Your goal is to construct SQL queries that accurately retrieve the data required to answer the Current Query.

**Output Format**

```json
{
    "query": "<current query>",
    "sql_cot_reasoning": "<step-by-step, CoT, reasoning behind crafting the SQL query>",
    "sql_query": "<a one-line SQL query to retrieve the required information>"
}
```

## Database and Table

{
  "question": "深科技在2021年12月24日的交易数据如何?具体包括收盘价、成交量、换手率，保留2位小数。",
  "database": "AStockMarketQuotesDB",
  "table": "QT_StockPerformance"
}

## Table-Column Schema

| table_name| column_name| column_description| 注释| Annotation |
|---|---|---|---|---|
| QT_StockPerformance | ID | ID | | |
| QT_StockPerformance | InnerCode| 证券内部编码| 证券内部编码（InnerCode）：与“证券主表（SecuMain）”中的“证券内部编码（InnerCode）”关联，得到证券的交易代码、简称等。| Security Internal Code (InnerCode): Associated 